In [1]:
import pandas as pd
import re

In [9]:
# https://www.ndis.gov.au/ - Data Source
ndis_df = pd.read_excel('../../../data/provider_list.xlsx', sheet_name = "Provider_by_Group_NT")

In [10]:
ndis_df.head(2)

,List of Registered Providers in Northern Territory (NT) by Registration Group as at 29 March 2021,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,Registration_Group,Registered_Provider_Name,Head_Office_Location,PrvdrPhnNmbr,Website,Email_Address,Head_Office_Address,PostCd,NaN
1,Accommodation/Tenancy,1 Primary Care Services Pty Ltd,North Strathfield,0460 297 297,www.primarycareservices.com.au,raj@primarycareservices.com.au,"9, George Street, North Strathfield, NSW 2137",2137,1.0


In [166]:
addresses = ndis_df["Unnamed: 6"][1:]
addresses.reset_index(drop=True, inplace=True)
addresses[10]

'9 Farm Cove St, Gregory Hills, NSW 2557'

In [56]:
## Get the Index file to GNAF
index_df = pd.read_parquet("../../../data/Master/New/index.parquet")

In [167]:
index_df.head(3)

,IDX,STREET_NAME,STREET_TYPE_CODE,LOCALITY_NAME,STATE,POSTCODE,FILE_NAME,ADDRESS_COUNT,ADDRESS
0,1,GRAZIER,LANE,BELCONNEN,ACT,2617,ACT-1.parquet,1917,GRAZIER LANE BELCONNEN ACT 2617
1,2,NORTHBOURNE,AVENUE,BRADDON,ACT,2612,ACT-1.parquet,1704,NORTHBOURNE AVENUE BRADDON ACT 2612
2,3,IRVING,STREET,PHILLIP,ACT,2606,ACT-1.parquet,1121,IRVING STREET PHILLIP ACT 2606


In [20]:
street_code_dict = {
        'ALLY': 'ALLEY',
         'ARC': 'ARCADE',
         'AVE': 'AVENUE',
         'AV': 'AVENUE',
         'BVD': 'BOULEVARD',
         'BYPA': 'BYPASS',
         'CCT': 'CIRCUIT',
         'CL': 'CLOSE',
         'CRN': 'CORNER',
         'CT': 'COURT',
         'CRES': 'CRESCENT',
         'CDS': 'CUL-DE-SAC',
         'DR': 'DRIVE',
         'ESP': 'ESPLANADE',
         'GRN': 'GREEN',
         'GR': 'GROVE',
         'HWY': 'HIGHWAY',
         'JNC': 'JUNCTION',
         'LANE': 'LANE',
         'LINK': 'LINK',
         'MEWS': 'MEWS',
         'PDE': 'PARADE',
         'PL': 'PLACE',
         'RDGE': 'RIDGE',
         'RD': 'ROAD',
         'SQ': 'SQUARE',
         'ST': 'STREET',
         'TCE': 'TERRACE'
}

In [118]:
def split_address(adr):   
    while True:
        match = re.search("[0-9]+[a-zA-Z,]*\s", adr)
        if not match: break
        pos = match.span()[1]    
        adr = adr[pos:]
    return adr.upper()    

In [128]:
def clean_address(adr):
    ## Clean the addresses - remove comma from "Unity 415, 21 Buchanan Street, West End, QLD 410"
    clean_adr = re.sub(r"((?:NSW|VIC|QLD|TAS|WA|SA|NT|ACT))[,\s]*([0-9]{4})$", r"\1 \2",adr)
    clean_adr = clean_adr.replace(",", "").strip().upper()
    return clean_adr

In [139]:
def filter_state_poscode(state, postcode):

    ## Firstly, Filter for the rows from the Index File based on same Postcode and State
    pocd_st_df = index_df[(index_df.POSTCODE == postcode) & (index_df.STATE == state)]
    if pocd_st_df.empty: return clean_adr
    return pocd_st_df

In [170]:
def get_filtered_suburb_list(pocd_st_df):
    
    ## Secondly, Filter further for same Suburbs, get the Distinct Suburb Name List first - "LOCALITY_NAME"
    suburb_list = pocd_st_df.LOCALITY_NAME.unique()
    suburbs = [sub for sub in suburb_list if sub in clean_adr]
    #### if suburbs length is zero ????
    return suburbs 

In [159]:
def filter_street(pocd_st_df, index, street_string):
    
    ## Thirdly, Filter further by Street Name within the filtered Suburb
    subb_df = pocd_st_df[index]
    unique_street_name_list = subb_df.STREET_NAME.unique()
    street_name = [street_name for street_name in unique_street_name_list if street_name in street_string] 

    if street_name: # if street name does not exist in the data frame, suburbs maybe incorrect
        st_df = subb_df[subb_df.STREET_NAME.isin(street_name)] 
    
    else:
        # find within the same post code and state again for street name
        street_name = [street_name for street_name in unique_street_name_list if street_name in street_string]
        st_df = pocd_st_df[pocd_st_df.STREET_NAME.isin(street_name)] 
    
    return st_df

In [174]:
def match_state_postcode_suburb_street_in_index_file(clean_adr):
     
    clean_adr = split_address(clean_adr)
    ## Search State and Postcode and extract - example QLD 410
    match = re.search("\s((?:NSW|VIC|QLD|TAS|WA|SA|NT|ACT))\s([0-9]{4})$", clean_adr)
    if not match: return clean_adr 
    state, postcode = match.group(1), match.group(2)
      
    ## Firstly, Filter for the rows from the Index File based on same Postcode and State
    pocd_st_df = filter_state_poscode(state, postcode)
    if pocd_st_df.empty: return clean_adr
      
    ## Secondly, Filter further for same Suburbs, get the Distinct Suburb Name List first - "LOCALITY_NAME"
    suburbs = get_filtered_suburb_list(pocd_st_df)
    if not suburbs : return clean_adr
    
    # Get the boolean array - of rows matching (LOCALITY_NAME) with filtered suburbs in the above list
    index = pocd_st_df.LOCALITY_NAME.str.contains('|'.join(suburbs))
    if not any(index): return clean_adr
    
    # Find Street and Street code from clean_adr to further matching by street name
    match = re.search(f"(?:{'|'.join(suburbs)})\s+{state}\s+{postcode}", clean_adr) # sample match output - <re.Match object; span=(29, 46), match='WEST END QLD 4101'>
    if not match: return clean_adr
    # Extract (unit, street) from the whole address string - UNIT 415 21 ABC STREET  from "UNIT 415 21 ABC STREET WEST END QLD 4101"
    street_string = clean_adr[:match.span()[0]]     

    ## Thirdly, Filter further by Street Name within the filtered Suburb
    st_df = filter_street(pocd_st_df, index, street_string)
    
    if st_df.empty: return clean_adr 
    
    return st_df

In [178]:
def match_address(adr):  
    
    ## Initial cleaning
    clean_adr = clean_address(adr)
    ## Get the narrowed down search state, postcode --> suburb --> street
    df = match_state_postcode_suburb_street_in_index_file(clean_adr)
    
    print("df", df)
    match = re.search(f"(?:({'|'.join(df.STREET_NAME.unique())}))\s(.*)\s((:?{'|'.join(df.LOCALITY_NAME.unique())}))", 
            clean_adr)
    
    if not match: 
        match = re.search(f"(?:({'|'.join(df.STREET_NAME.unique())}))\s(\S*)\s",clean_adr)
        if not match: return clean_adr
        
        street_name = match.group(1)
        street_code = match.group(2)
        
        match = re.search(f'{street_code}\s(.+)\s{state}', clean_adr)
        # Suburb maybe incorrectly entered
        suburb = match.group(1) if match else None      
    else:
        street_name, street_code, suburb = (match.group(i+1) for i in range(3))   
        
    
    if suburb:
        # Get the index of filter based on Street and Suburb name
        index = (df.STREET_NAME == street_name) & (df.LOCALITY_NAME == suburb)
        # If no exact match found, try to find matching string with Street name and Suburb
        index = index if any(index) else (df.STREET_NAME.str.contains(street_name)) & (df.LOCALITY_NAME.str.contains(suburb))    
        
        # if there is no matching suburbs found, use street name only to filter
        df = df[index] if any(index) else df[(df.STREET_NAME.str.contains(street_name))]
    
    else:
        df = df[(df.STREET_NAME == street_name)]
    
    if df.empty: return clean_adr
    if df.shape[0] == 1: return df.ADDRESS.iloc[0]   
    
    # Covert street code to long form postcode
    street_code = street_code_dict[street_code] if street_code in street_code_dict else street_code
    df = df[df.STREET_TYPE_CODE == street_code]
    
    if df.empty: return clean_adr
    if df.shape[0] == 1: return df.ADDRESS.iloc[0]
    return clean_adr  

In [181]:
##### Testing ######
adr = "14/595 Collins Street, Melbourne, VIC 3000"
df = match_state_postcode_suburb_street_in_index_file(adr)
df

'COLLINS STREET, MELBOURNE, VIC 3000'